In [ ]:
#I am having an issue replicating this notebook. The main issue being 'extract_data_from_enzymeml' for visualization

import jax

from catalax import Model, optimize, visualize
from utils import extract_data_from_enzymeml, create_title

jax.config.update("jax_enable_x64", True)

In [ ]:
model = Model.load("C:/Users/danic/OneDrive/Desktop/Masters Thesis/Kyle time arrays/final_initials - Copy.json")
model

In [ ]:
# Extract data from EnzymeML
species_mapping = {
    "s0": "s0",
    "s1": "s1",
    "s2": "s2",
    "s3": "s3",
}

initial_conditions, times, data = extract_data_from_enzymeml(
    path="C:/Users/danic/OneDrive/Desktop/Masters Thesis/PGM-ENO-(Kyle)_timescaled (2).omex",
    model=model,
    species_mapping=species_mapping,
)

# Remove the high PGME data
initial_conditions = initial_conditions
data = data
times = times

In [ ]:
# Perform the optimization
result, new_model = optimize(
    model,
    initial_conditions[0:-1],
    data[0:-1],
    times[0:-1],
    global_upper_bound=1e5,
    global_lower_bound=1e-6,
    method="bfgs"
)

In [ ]:
# Visualize the results
aic = new_model.calculate_aic(data[0:-1], initial_conditions[0:-1], times[0:-1]) / data[0:-1].shape[-1]
f = visualize(
    model=new_model,
    data=data[0:-1],
    times=times[0:-1],
    initial_conditions=initial_conditions[0:-1],
    colors={"s0": "green", "s1": "blue", "s2": "orange", "s3": "red"},
    heading_fontsize=10,
    save="C:/Users/danic/OneDrive.Desktop/Masters Thesis",
    title=create_title(
        model_name="Model 07",
        aic=aic,
        sub="...",
    ),
)

In [ ]:
# Perform the optimization
result, new_model = optimize(
    model,
    initial_conditions,
    data,
    times,
    global_upper_bound=1e5,
    global_lower_bound=1e-6,
    method="bfgs"
)

In [ ]:
# Visualize the results
aic = new_model.calculate_aic(data, initial_conditions, times) / data.shape[-1]
f = visualize(
    model=new_model,
    data=data,
    times=times,
    initial_conditions=initial_conditions,
    colors={"s0": "green", "s1": "blue", "s2": "orange", "s3": "red"},
    heading_fontsize=10,
    save="./plots/model_07_fit",
    title=create_title(
        model_name="Model 07",
        aic=aic,
        sub="......",
    ),
)